# tf-mme

In [1]:
import os
import sagemaker
from sagemaker import get_execution_role
import pandas as pd

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name
bucket = sagemaker_session.default_bucket()

In [4]:
!mkdir -p multi/model1
!mkdir -p multi/model2
!tar xvf model1.tar.gz -C ./multi/model1
!tar xvf model2.tar.gz -C ./multi/model2

1611669077/
1611669077/saved_model.pb
1611669077/variables/
1611669077/variables/variables.data-00001-of-00002
1611669077/variables/variables.index
1611669077/variables/variables.data-00000-of-00002
1611669077/
1611669077/saved_model.pb
1611669077/variables/
1611669077/variables/variables.data-00001-of-00002
1611669077/variables/variables.index
1611669077/variables/variables.data-00000-of-00002


In [12]:
!tar -czvf multi.tar.gz multi/

multi/
multi/model1/
multi/model1/1611669077/
multi/model1/1611669077/variables/
multi/model1/1611669077/variables/variables.data-00001-of-00002
multi/model1/1611669077/variables/variables.data-00000-of-00002
multi/model1/1611669077/variables/variables.index
multi/model1/1611669077/saved_model.pb
multi/model2/
multi/model2/1611669077/
multi/model2/1611669077/variables/
multi/model2/1611669077/variables/variables.data-00001-of-00002
multi/model2/1611669077/variables/variables.data-00000-of-00002
multi/model2/1611669077/variables/variables.index
multi/model2/1611669077/saved_model.pb


In [13]:
# 创建模型，不提供inference.py

from sagemaker.tensorflow import TensorFlowModel

env = {
  'SAGEMAKER_TFS_DEFAULT_MODEL_NAME': 'model1'
}

iris_model = TensorFlowModel(model_data='s3://your_bucket_name/lb-mme-tf/multi.tar.gz',
                        role=role,
                        framework_version='1.15.2',
                        env=env
                        )

In [14]:
# 部署

predictor = iris_model.deploy(initial_instance_count=1, instance_type='ml.g4dn.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


------!

In [21]:
from sagemaker.tensorflow import TensorFlowPredictor

model2_predictor = TensorFlowPredictor('tensorflow-inference-2022-04-27-15-04-02-524', model_name='model2')

In [29]:
from sagemaker.tensorflow import TensorFlowPredictor

model1_predictor = TensorFlowPredictor('tensorflow-inference-2022-04-27-15-04-02-524', model_name='model1')

In [35]:
# 将iris_test.csv数据中的前五行数据取出，构造classify接口所能接受的examples

df = pd.read_csv('iris_test.csv', index_col=False)
examples = []
for i in df.index[:5]:
    row = {col: df.loc[i, col] for col in df.columns[:4]}
    examples.append(row)
data1 = {'examples': examples}
data1

{'examples': [{'0': 6.1, '1': 2.8, '2': 4.7, '3': 1.2},
  {'0': 5.7, '1': 3.8, '2': 1.7, '3': 0.3},
  {'0': 7.7, '1': 2.6, '2': 6.9, '3': 2.3},
  {'0': 6.0, '1': 2.9, '2': 4.5, '3': 1.5},
  {'0': 6.8, '1': 2.8, '2': 4.8, '3': 1.4}]}

In [17]:
# 推理

predictor.classify(data1)

{'results': [[['0', 0.0284139793], ['1', 0.639631271], ['2', 0.331954837]],
  [['0', 0.961801767], ['1', 0.0314502306], ['2', 0.0067479061]],
  [['0', 0.0027867991], ['1', 0.0766616166], ['2', 0.920551538]],
  [['0', 0.0266317707], ['1', 0.590011418], ['2', 0.38335681]],
  [['0', 0.0254174341], ['1', 0.60687983], ['2', 0.367702723]]]}

In [31]:
model2_predictor.classify(data1)

{'results': [[['0', 0.0284139793], ['1', 0.639631271], ['2', 0.331954837]],
  [['0', 0.961801767], ['1', 0.0314502306], ['2', 0.0067479061]],
  [['0', 0.0027867991], ['1', 0.0766616166], ['2', 0.920551538]],
  [['0', 0.0266317707], ['1', 0.590011418], ['2', 0.38335681]],
  [['0', 0.0254174341], ['1', 0.60687983], ['2', 0.367702723]]]}

In [30]:
model1_predictor.classify(data1)

{'results': [[['0', 0.0284139793], ['1', 0.639631271], ['2', 0.331954837]],
  [['0', 0.961801767], ['1', 0.0314502306], ['2', 0.0067479061]],
  [['0', 0.0027867991], ['1', 0.0766616166], ['2', 0.920551538]],
  [['0', 0.0266317707], ['1', 0.590011418], ['2', 0.38335681]],
  [['0', 0.0254174341], ['1', 0.60687983], ['2', 0.367702723]]]}

In [ ]:
while True:
    res = model1_predictor.classify(data1)